In [1]:
# Get the data
from sklearn.datasets import load_digits
import numpy as np

digit = load_digits()
X, y = digit.images, digit.target

print(f'{X.shape = }')
print(f'{y.shape = }')

X.shape = (1797, 8, 8)
y.shape = (1797,)


In [2]:
# Partition the training and testing data
from neural_network.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print(f'{X_train.shape = }')
print(f'{y_train.shape = }')
print(f'{X_test.shape = }')
print(f'{y_test.shape = }')

X_train.shape = (1348, 8, 8)
y_train.shape = (1348,)
X_test.shape = (449, 8, 8)
y_test.shape = (449,)


In [3]:
# Expand dimensions for convolutions
X_train = X_train[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]

print(f'{X_train.shape = }')
print(f'{X_test.shape = }')

X_train.shape = (1348, 1, 8, 8)
X_test.shape = (449, 1, 8, 8)


In [4]:
# One Hot Encode the labels
from neural_network.preprocess import OneHotEncoder

labels = np.unique(y)
print(f'\n{labels = }\n')  # Unique labels

encoder = OneHotEncoder().fit(labels)
y_train_encoded = encoder.transform(y_train)
y_test_encoded = encoder.transform(y_test)

print(f'{y_train_encoded.shape = }')  # Encoded training labels
print(f'{y_test_encoded.shape = }')   # Encoded testing labels


labels = array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

y_train_encoded.shape = (1348, 10)
y_test_encoded.shape = (449, 10)


In [5]:
# Define a sequential model
from neural_network.model import Sequential

net = Sequential()

In [6]:
#  Add layers to the model
from neural_network.layers import Dense, Flatten, Convolutional

net.add(
    Convolutional(25, 3, input_shape=(1, 8, 8), activation='tanh'),
    Flatten(),
    Dense(128, activation='tanh'),
    Dense(64, activation='leaky_relu'),
    Dense(10, activation='softmax')
)

In [7]:
# Compile and build  the model
net.compile(cost='crossentropy', metrics=['accuracy_score'])

In [8]:
# Initialize cross validator
from neural_network.model_selection import KFold

kf = KFold(n_splits=5)

In [9]:
# Train the model with Cross Validation
for train, validate in kf.split(X_train):
    net.fit(
        X_train[train],
        y_train_encoded[train],
        epochs=50,
        batch_size=128,
        validation_data=(X_train[validate], y_train_encoded[validate]),
        verbose=False # The output is huge otherwise
    )


  Validation loss: 3.0424 accuracy_score: 0.2119

  Validation loss: 2.786 accuracy_score: 0.3717

  Validation loss: 2.5592 accuracy_score: 0.5316

  Validation loss: 2.361 accuracy_score: 0.6766

  Validation loss: 2.1818 accuracy_score: 0.7472

  Validation loss: 2.0259 accuracy_score: 0.777

  Validation loss: 1.8966 accuracy_score: 0.7955

  Validation loss: 1.7671 accuracy_score: 0.8178

  Validation loss: 1.6509 accuracy_score: 0.8439

  Validation loss: 1.5528 accuracy_score: 0.8587

  Validation loss: 1.459 accuracy_score: 0.8662

  Validation loss: 1.3676 accuracy_score: 0.8773

  Validation loss: 1.2872 accuracy_score: 0.8848

  Validation loss: 1.213 accuracy_score: 0.8959

  Validation loss: 1.1511 accuracy_score: 0.9033

  Validation loss: 1.0968 accuracy_score: 0.9108

  Validation loss: 1.0431 accuracy_score: 0.9108

  Validation loss: 0.9941 accuracy_score: 0.9182

  Validation loss: 0.9526 accuracy_score: 0.9182

  Validation loss: 0.9114 accuracy_score: 0.9182

  Va

In [10]:
# Make predictions on the testing data
predictions = net.predict(X_test, classify=True)

In [11]:
# The confusion matrix
from neural_network.metrics import confusion_matrix
cmat = confusion_matrix(y_test_encoded, predictions) 
print(cmat)

[[45  0  0  0  0  0  0  0  0  0]
 [ 0 41  0  0  0  0  0  0  3  0]
 [ 0  1 45  0  0  0  0  0  0  0]
 [ 0  0  0 43  0  0  0  1  3  0]
 [ 0  0  0  0 31  0  1  0  0  0]
 [ 0  0  0  0  0 58  0  0  0  1]
 [ 0  1  0  0  0  0 40  0  1  0]
 [ 0  0  0  0  0  0  0 33  0  0]
 [ 0  4  0  0  0  0  1  0 44  0]
 [ 0  1  0  1  0  0  0  0  1 49]]


In [12]:
# Accuracy of the model
from neural_network.metrics import accuracy_score
acc = accuracy_score(y_test_encoded, predictions)
print(f'Accuracy: {np.around(acc, 3) * 100}%')

Accuracy: 95.5%


In [13]:
# Accuracy by label
from neural_network.metrics import accuracy_by_label
acc_by_label = accuracy_by_label(y_test_encoded, predictions)
print('Accuracy by Label:', np.around(acc_by_label, 3))

Accuracy by Label: [1.    0.854 1.    0.977 1.    1.    0.952 0.971 0.846 0.98 ]
